1. SUMMARY FROM PDF IMAGES USING GROQ
2. COLPALI VECTORSTORE TO INDEX 

In [59]:
from byaldi import RAGMultiModalModel

INDEX_NAME = "finance_data"
RAG = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2")

RAG.index(
    input_path="finance/", # The path to your documents
    index_name=INDEX_NAME, # The name you want to give to your index. It'll be saved at `index_root/index_name/`.
    store_collection_with_index=True, # Whether the index should store the base64 encoded documents.
    # doc_ids=[0, 1, 2], # Optionally, you can specify a list of document IDs. They must be integers and match the number of documents you're passing. Otherwise, doc_ids will be automatically created.
    # metadata=[{"author": "John Doe", "date": "2021-01-01"}], # Optionally, you can specify a list of metadata for each document. They must be a list of dictionaries, with the same length as the number of documents you're passing.
    overwrite=True # Whether to overwrite an index if it already exists. If False, it'll return None and do nothing if `index_root/index_name` exists.
)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:4531: RuntimeWarning: coroutine 'get_summary' was never awaited
  gc.collect()
/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:4531: RuntimeWarning: coroutine 'AsyncCompletions.create' was never awaited
  gc.collect()
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.18it/s]


Indexing file: finance/jll-asia-pacific-capital-tracker-3q23 (1).pdf
Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of document 0 

{0: 'finance/jll-asia-pacific-capital-tracker-3q23 (1).pdf',
 1: 'finance/APAC_Tale of HK SAR _ SNG Report_short version_FINAL.pdf'}

In [4]:
search_index = RAG.from_index(INDEX_NAME)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.27it/s]


You are using in-memory collection. This means every image is stored in memory.
You might want to rethink this if you have a large collection!
Loaded 70 images from 1 JSON files.


/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/byaldi/colpali.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.ext

In [5]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(), override=True)

In [6]:
res = search_index.search(query="How many companies are leaving Hong Kong",k=1)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


## PDF TO IMAGES

In [58]:
s = "He.loo.asd"
s.rsplit(".",1)

['He.loo', 'asd']

In [41]:
# import os
# from pdf2image import convert_from_path
# from PIL import Image
# import multiprocessing

# def process_pdf(args):
#     pdf_file, input_dir, output_dir = args
#     pdf_path = os.path.join(input_dir, pdf_file)
#     curr_output_dir = os.path.join(output_dir, pdf_file[:-4])  # Remove .pdf extension
#     os.makedirs(curr_output_dir, exist_ok=True)
    
#     images = convert_from_path(pdf_path)
#     for i, image in enumerate(images):
#         image_path = os.path.join(curr_output_dir, f'page_{i+1}.png')
#         image.save(image_path, 'PNG')
    
#     print(f"Processed: {pdf_file}")

# def pdf_to_png_parallel(input_dir, output_dir):
#     # Create output directory if it doesn't exist
#     os.makedirs(output_dir, exist_ok=True)

#     # Get list of PDF files
#     pdf_files = [f for f in os.listdir(input_dir) if f.endswith('.pdf')]

#     # Prepare arguments for multiprocessing
#     args = [(pdf_file, input_dir, output_dir) for pdf_file in pdf_files]

#     # Use all available CPU cores
#     num_processes = multiprocessing.cpu_count()

#     # Create a pool of worker processes
#     with multiprocessing.Pool(processes=num_processes) as pool:
#         # Map the process_pdf function to all arguments
#         pool.map(process_pdf, args)

# if __name__ == '__main__':
#     input_dir = 'finance'  # Directory containing PDF files
#     output_dir = 'output_png2_files'
#     pdf_to_png_parallel(input_dir, output_dir)

In [56]:
from groq import AsyncGroq
import base64
import os
import asyncio
from typing import List, Optional, Annotated
from pydantic import BaseModel
from io import BytesIO
from PIL import Image



client = AsyncGroq(api_key=os.environ['GROQ_API_KEY'])

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Data model for LLM to generate
class SummaryResponse(BaseModel):
    summary: Annotated[str, "Summary of the image in 2-3 lines that briefly describes the image"]
    keywords: Annotated[List[str], "List of keywords mentioned in the page"]


async def get_summary(base64_enc: str) -> SummaryResponse:
    completion = await client.chat.completions.create(
    model="llama-3.2-11b-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Summarize the image in 2-3 lines"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(base64_enc)}",
                    },
                },
            ],
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    # response_format={"type": "json_object"},
    stop=None,
    )
    # return SummaryResponse.model_validate_json(chat_completion.choices[0].message.content)
    return completion.choices[0].message.content




recipe = await get_summary("output_png2_files/jll-asia-pacific-capital-tracker-3q23 (1)/page_4.png")
# if __name__ == '__main__':
    # summary = asyncio.run(get_summary(data['base_64']))
# print_recipe(recipe)

In [57]:
recipe

'The image depicts a bar graph titled "Currency hedging impact", which visually represents the effects of currency hedging transactions on cross-currency returns. The graph features three sections: "Depreciation of APAC currencies against USD", "Transaction volume changes by geography for 2012 vs. 2012", and "Policy rates by geography", set against a background of informative text in narrative form.'

In [3]:
import os

for d in os.listdir("visualfinanceagent/vectordb/output_imgs/jll-asia-pacific-capital-tracker-3q23 (1)"):
    if d.endswith(".pdf"):
        metadata_path = d.rsplit(".")[0] + ".json"
        

page_8.png
page_29.json
page_1.json
page_37.json
page_41.png
page_32.png
page_2.json
page_13.png
page_20.png
page_41.json
page_9.json
page_12.png
page_22.json
page_18.png
page_35.png
page_19.png
page_32.json
page_30.json
page_33.png
page_7.json
page_40.png
page_34.json
page_38.json
page_3.png
page_24.json
page_30.png
page_27.json
page_25.png
page_10.png
page_11.png
page_39.json
page_14.json
page_16.png
page_21.json
page_21.png
page_31.json
page_39.png
page_17.png
page_27.png
page_33.json
page_4.json
page_22.png
page_28.png
page_16.json
page_37.png
page_14.png
page_3.json
page_11.json
page_31.png
page_5.json
page_36.png
page_26.png
page_23.png
page_35.json
page_26.json
page_6.png
page_7.png
page_6.json
page_38.png
page_19.json
page_15.png
page_2.png
page_5.png
page_1.png
page_24.png
page_36.json
page_18.json
page_8.json
page_13.json
page_40.json
page_10.json
page_29.png
page_9.png
page_17.json
page_34.png
page_20.json
page_28.json
page_12.json
page_4.png
page_25.json
page_15.json
page_2

In [8]:
# import os
# import shutil

# def organize_files(source_dir):
#     # Create subdirectories
#     png_dir = os.path.join(source_dir, "PNG")
#     json_dir = os.path.join(source_dir, "JSON")
    
#     os.makedirs(png_dir, exist_ok=True)
#     os.makedirs(json_dir, exist_ok=True)

#     # Iterate through files in the source directory
#     for filename in os.listdir(source_dir):
#         file_path = os.path.join(source_dir, filename)
        
#         # Skip if it's not a file
#         if not os.path.isfile(file_path):
#             continue
        
#         # Move PNG files
#         if filename.lower().endswith('.png'):
#             shutil.move(file_path, os.path.join(png_dir, filename))
#             print(f"Moved {filename} to PNG directory")
        
#         # Move JSON files
#         elif filename.lower().endswith('.json'):
#             shutil.move(file_path, os.path.join(json_dir, filename))
#             print(f"Moved {filename} to JSON directory")

# # Usage
# source_directory = 'visualfinanceagent/vectordb/output_imgs_2/APAC_Tale of HK SAR _ SNG Report_short version_FINAL'
# organize_files(source_directory)

In [16]:
from byaldi import RAGMultiModalModel

rag_index = RAGMultiModalModel.from_index(index_path="finance_data")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]


You are using in-memory collection. This means every image is stored in memory.
You might want to rethink this if you have a large collection!
Loaded 70 images from 1 JSON files.


/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/byaldi/colpali.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.ext

In [17]:
res = rag_index.search(
    query="What did Hong Kong do better than Singapore?",
    k = 1
)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [5]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda','trust_remote_code':True}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from langchain.schema import Document
import os
import json

docs = []
path = "visualfinanceagent/vectordb/output_imgs_2"
for dir in os.listdir(path):
    pdfs = os.path.join(path,dir)
    for json_path in os.listdir(os.path.join(pdfs,"JSON")):
        with open(os.path.join(pdfs,"JSON",json_path), 'r') as file:
            data = json.load(file)
        docs.append(Document(page_content=data['summary'],metadata={"filename":dir,"page_num":json_path}))

In [11]:
db = FAISS.from_documents(docs, hf)

In [12]:
retriever = db.as_retriever()

In [14]:
docs = retriever.invoke("Difference between hong kong and singapore?")

docs

[Document(metadata={'filename': 'APAC_Tale of HK SAR _ SNG Report_short version_FINAL', 'page_num': 'page_14.json'}, page_content="Hong Kong SAR maintains a deeper finance talent pool, while Singapore boasts stronger science and high value workers. In 2022, Hong Kong SAR's total employment and hiring surged ahead of Singapore's. The talent pool in both Hong Kong SAR and Singapore will continue to deepen in the long run."),
 Document(metadata={'filename': 'APAC_Tale of HK SAR _ SNG Report_short version_FINAL', 'page_num': 'page_15.json'}, page_content='There are significant disparities between foreign workers from Singapore and Hong Kong SAR. The number of Hong Kong SAR workers in Singapore and vice versa have both seen declines since 2018, with Singapore still having a higher number than Hong Kong SAR. \n\nPlease let me know if I can help with anything else.'),
 Document(metadata={'filename': 'APAC_Tale of HK SAR _ SNG Report_short version_FINAL', 'page_num': 'page_5.json'}, page_conte

In [1]:
from pipeline import VisionFinancePipeLine

vfsp = VisionFinancePipeLine()

/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s]


Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.32it/s]


You are using in-memory collection. This means every image is stored in memory.
You might want to rethink this if you have a large collection!
Loaded 70 images from 1 JSON files.


/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/byaldi/colpali.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.ext

In [3]:
# vfsp.vision_index.search("Hong Kong",k=2)

In [ ]:
vfsp.summary_index.invoke()